# Tutorial For ppo_torch

In this notebook, I will go through how PPO works and how it is implemented in the `ppo_torch` library. This notebook does not contain any code for training, in the hope that the training loop is self-explanitory. Here is a list of what this notebook will go through:

- [Creating multi-layer perceptrons](#mlp)
- [Creating Actors](#actors_create)
- [Updating Actors](#actors_update)
- [Creating Critics](#critics_create)
- [Updating Critics](#critics_update)
- [Replay Buffer](#replay_buffer)

In [10]:
import numpy as np
import torch
from torch import nn

from actor import ActorContinuous, ActorDiscrete
from critic import Critic
from mlp import mlp

## Creating Multi-Layer Perceptrons
<a id='mlp'></a>

The function that creates a multi-layer perceptron, `mlp()`, is defined in `mlp.py`. It takes in two lists

- `feature_sizes`: a list of the size of each layer of the mlp
- `activation`: a list of the activation functions for each layer of the mlp

For example, if the MLP has input size of 8, output size of 4 and 2 hidden layers of size 16, we will define `nn_size = [8, 16, 16, 4]`. Since there are three layers:

- Input Layer (8, 16)
- Hidden Layer (16, 16)
- Output Layer (16, 4)

and we set all layers to have ReLU activation, we can define `activations = [nn.ReLU, nn.ReLU, nn.ReLU]`.

In [11]:
nn_sizes = [8, 16, 16, 4]
activations = [nn.ReLU, nn.ReLU, nn.ReLU]
mlp_model = mlp(nn_sizes, activations)
print(mlp_model)

Sequential(
  (0): Linear(in_features=8, out_features=16, bias=True)
  (1): ReLU()
  (2): Linear(in_features=16, out_features=16, bias=True)
  (3): ReLU()
  (4): Linear(in_features=16, out_features=4, bias=True)
  (5): ReLU()
)


## Create Actors
<a id='actors_create'></a>

We can set that the `obs_dim = 8` and `act_dim = 4`, we also set the hidden layer have both the `in_feature` and `out_feature` to be 16. Thus, we have `hidden_dim = [16, 16]`. Additionally, we use ReLU for all of the activations, then we have `activations = [nn.ReLU, nn.ReLU, nn.ReLU]`.

Using this set up, we can define actors for both continuous and discrete action spaces. First, we define a discrete actor using the `ActorDiscrete` class, which is defined in `actor.py`. 

In [12]:
obs_dim = 8
act_dim = 4
hidden_dim = [16, 16]
activations = [nn.ReLU, nn.ReLU, nn.ReLU]

actor_d = ActorDiscrete(obs_dim, act_dim, hidden_dim, activations)

In [13]:
print(actor_d.net)

Sequential(
  (0): Linear(in_features=8, out_features=16, bias=True)
  (1): ReLU()
  (2): Linear(in_features=16, out_features=16, bias=True)
  (3): ReLU()
  (4): Linear(in_features=16, out_features=4, bias=True)
  (5): ReLU()
)


Here we can see that the neural network is defined as `actor_d.net`. Next, we see what is the output of `actor_d.forward(obs)`, note this can be directly called using `actor_d(obs)`.

In [14]:
obs = torch.ones(1, 8)
pi, _ = actor_d(obs)
print(pi)
print(pi.sample())

Categorical(logits: torch.Size([1, 4]))
tensor([2])


We see that the output is not the action, but a probability distribution. Here for discrete actions, the output of `actor_d` is a `Categorical` distribution, we can sample from this distribution `pi` using `pi.sample()`. The output of this sampling is the index of the sampled action, here the output will only be 0, 1, 2, 3.

Next we do the same thing for the actor for continuous action spaces using the `ActorContinuous` class.

In [15]:
obs_dim = 8
act_dim = 4
hidden_dim = [16, 16]
activations = [nn.ReLU, nn.ReLU, nn.ReLU]

actor_c = ActorContinuous(obs_dim, act_dim, hidden_dim, activations)

In [16]:
print(actor_c.net)

Sequential(
  (0): Linear(in_features=8, out_features=16, bias=True)
  (1): ReLU()
  (2): Linear(in_features=16, out_features=16, bias=True)
  (3): ReLU()
  (4): Linear(in_features=16, out_features=4, bias=True)
  (5): ReLU()
)


In [17]:
obs = torch.ones(1, 8)
pi, _ = actor_c(obs)
print(pi)
print(pi.sample())

MultivariateNormal(loc: torch.Size([1, 4]), covariance_matrix: torch.Size([1, 4, 4]))
tensor([[-0.1981,  0.1785, -0.1717,  0.3097]])


Since here we have a continuous action space, the output of `actor_c.forward()` will be a normal distribution with mean `pi.loc`, which is the same as `actor_c.net(obs)` and a fixed covariance matrix `pi.covariance_matrix`.

In [23]:
pi.loc, actor_c.net(obs), pi.covariance_matrix

(tensor([[0.0000, 0.0000, 0.0093, 0.3460]], grad_fn=<ExpandBackward>),
 tensor([[0.0000, 0.0000, 0.0093, 0.3460]], grad_fn=<ReluBackward0>),
 tensor([[[0.0400, 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0400, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0400, 0.0000],
          [0.0000, 0.0000, 0.0000, 0.0400]]]))

## Updating Actors
<a id='actors_update'></a>

### Actors Loss Function
The actor is updated by maximizing the PPO-Clip objective

$$
\DeclareMathOperator*{\argmax}{\arg\!\max}
\theta_{k+1} = \argmax_\theta\frac{1}{|\mathcal{D}_k|T}\sum_{\tau\in\mathcal{D}_k}\sum_{t=0}^T\min\Bigg(\frac{\pi_\theta(a_t\mid s_t)}{\pi_{\theta_k}(a_t\mid s_t)}A^{\pi_{\theta_k}}(a_t, s_t),\ g\Big(\epsilon, A^{\pi_{\theta_k}}(a_t, s_t)\Big)\Bigg)$$

This seems like a mouthful of symbols, let's dissect it so it makes sense. Here $|\mathcal{D}_k|$ represents the number of episodes used in optimizing the actor's weights, and $T$ is the number of step sizes for each episode. Thus, we can see the entire equation is simply saying the next weights $\theta_k$ is the set of weights that has the largest mean

$$\min\Bigg(\frac{\pi_\theta(a_t\mid s_t)}{\pi_{\theta_k}(a_t\mid s_t)}A^{\pi_{\theta_k}}(a_t, s_t),\ g\Big(\epsilon, A^{\pi_{\theta_k}}(a_t, s_t)\Big)\Bigg).$$

We can see that the policy gradient loss

$$\mathcal{L}_{PG} = \mathbb{E}_t\Bigg[\log\Big(\pi_\theta(a_t\mid s_t)\Big)A^{\pi_{\theta_k}}(a_t, s_t)\Bigg]$$

has the same gradient w.r.t $\theta$ as

$$\bar{\mathcal{L}}_{PPO} = \mathbb{E}_t\Bigg[\frac{\pi_\theta(a_t\mid s_t)}{\pi_{\theta_k}(a_t\mid s_t)}A^{\pi_{\theta_k}}(a_t, s_t)\Bigg].$$

Note $A^{\pi_{\theta_k}}(a_t, s_t)$, which is the generalized advantage estimation, is not a function of $\theta$, we will go into this later. We have

$$\nabla_\theta\log\Big(\pi_\theta(a_t\mid s_t)\Big)\Big|_{\theta_k} = \Bigg(\frac{\partial\log\Big(\pi_\theta(a_t\mid s_t)\Big)}{\partial\pi_\theta(a_t\mid s_t)}\frac{\partial\pi_\theta(a_t\mid s_t)}{\partial\theta}\Bigg)\Bigg|_{\theta_k} = \frac{\nabla_\theta\pi_\theta(a_t\mid s_t)}{\pi_\theta(a_t\mid s_t)}\Big|_{\theta_k} = \nabla_\theta\Big(\frac{\nabla_\theta\pi_\theta(a_t\mid s_t)}{\pi_{\theta_k}(a_t\mid s_t)}\Big)\Big|_{\theta_k}$$

The function $g(\epsilon, A)$ is defined as

$$g(\epsilon, A) = \begin{cases}
    (1+\epsilon)A & A \geq 0\\
    (1-\epsilon)A & A < 0
\end{cases}$$

The next policy will want to maximize the probability of state-action pairs with positive advantage and minimize the probability of state-action pairs with negative advantage. However, in both cases, PPO would not want to change the policy too much, hence using $g(\epsilon, A)$ to regulate this amount of change.

The loss function is defined in `ppo.py`, see the function `actor_loss()`.

## Creating Critics
<a id='critics_create'></a>

We can set the `obs_dim = 8`, since the output is the value function, it always has dimension 1. Similar to the actor, we set the `in_feature`'s and `out_feature`'s to be `hidden_dim = [16, 16]` and all activations to be `activations = [nn.ReLU, nn.ReLU, nn.ReLU]`. 

In [19]:
obs_dim = 8
hidden_dim = [16, 16]
activations = [nn.ReLU, nn.ReLU, nn.ReLU]
critic = Critic(obs_dim, hidden_dim, activations)

We can see that the output of the critic is simply a one dimension value. For both the continuous and discrete actor, the structure of their critic would be the same.

In [20]:
obs = torch.ones(1, 8)
val = critic(obs)
print(val)

tensor([[0.]], grad_fn=<ReluBackward0>)


## Updating Critics
<a id='critics_create'></a>

The critic is simply updated using the MSE loss between the value function estimation at each state $V_\phi(s_t)$ and its reward-to-go $\hat{R}(s_t)$ for one specific episode

$$
\DeclareMathOperator*{\argmin}{\arg\!\min}
\mathcal{L}_{VF} = \argmin_{\phi}\frac{1}{|\mathcal{D}_k|T}\sum_{\tau\in\mathcal{D}_k}\sum_{t=0}^T\Big(V_\phi(s_t) - \hat{R}(s_t)\Big)
$$

The loss function is defined in `ppo.py`, see the function `critic_loss()`.

## Replay Buffer
<a id='replay_buffer'></a>

The purpose of the replay buffer is to store the experiences for each episode. Instead of sampling a batch of data as in off-policy RL algorithms, PPO will use all of the data within the replay buffer to calculate a gradient estimation.

One of the calculations that are done in the replay buffer is obtaining the generalized advantage estimation, which is explained below.

### Generalized Advantage Estimation
The one step TD error at time step $t$ is defined as

$$\delta_t(s_t, a_t, s_{t+1}) = r_t + \gamma V^\pi(s_{t+1}) - V^\pi(s_t)$$

here the reward is a function of the current state $s_t$, the current action $a_t$ and the next state $s_{t+1}$. The advantage function is the expected TD error, where is taken w.r.t the system dynamics ([here is a nice article](http://boris-belousov.net/2017/08/10/td-advantage-bellman/))

$$A^\pi(s_t, a_t) = \mathbb{E}_{s_{t+1}\sim\mathcal{P}}\Big[r_t + \gamma V^\pi(s_{t+1})\Big] - V^\pi(s_t) = \mathbb{E}_{s_{t+1}\sim\mathcal{P}}[\delta_t].$$

And the generalized advantage estimation (GAE) is simply

$$A_{\gamma, \lambda}^\pi(s_t, a_t) = \sum_{l=0}^{H}(\gamma\lambda)^l\delta_{t+l}$$

where $H$ is the number of steps till the terminal step.